## Step 1: Prepare the Training and Evaluation Dataset

In [1]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)
(X.shape, y.shape)

((442, 10), (442,))

In [2]:
[(X[0], y[0]), (X[1], y[1])]

[(array([ 0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
         -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613]),
  151.0),
 (array([-0.00188202, -0.04464164, -0.05147406, -0.02632753, -0.00844872,
         -0.01916334,  0.07441156, -0.03949338, -0.06833155, -0.09220405]),
  75.0)]

## Step 2: Launching the Spark Session with Rikai Extension

In [3]:
from example import spark

:: loading settings :: url = jar:file:/Users/da/.cache/pants/named_caches/pex_root/installed_wheels/fcaa57f02b90be772d50778078fc41c3660d5a6c43218e45b2c2aef2ec8e9d58/pyspark-3.2.2-py2.py3-none-any.whl/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/da/.ivy2/cache
The jars for the packages stored in: /Users/da/.ivy2/jars
ai.eto#rikai_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c28e7f4c-0fd3-4982-96c1-f1051c324c13;1.0
	confs: [default]
	found ai.eto#rikai_2.12;0.1.14 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found com.typesafe.scala-logging#scala-logging_2.12;3.9.4 in central
	found org.slf4j#slf4j-api;1.7.30 in spark-list
	found org.mlflow#mlflow-client;1.21.0 in central
	found org.apache.logging.log4j#log4j-core;2.17.1 in central
:: resolution report :: resolve 207ms :: artifacts dl 10ms
	:: modules in use:
	ai.eto#rikai_2.12;0.1.14 from central in [default]
	com.typesafe.scala-logging#scala-logging_2.12;3.9.4 from central in [default]
	org.apache.logging.log4j#log4j-core;2.17.1 from central in [default]
	org.mlflow#mlflow-client;1.21.0 from central in [default]
	org.slf4j#slf4j-api;1.7.30 from spark-list in [default]
	org.

## Step 3: Training and Logging using MLflow

In [4]:
import getpass

import mlflow
import rikai_sklearn
from sklearn.linear_model import LinearRegression


mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)

# train a model
model = LinearRegression()
with mlflow.start_run() as run:
    ####
    # Part 1: Train the model and register it on MLflow
    ####
    model.fit(X, y)

    registered_model_name = f"{getpass.getuser()}_sklearn_lr"
    rikai_sklearn.log_model(
        model,
        registered_model_name=registered_model_name)


/Users/da/.cache/pants/named_caches/pex_root/venvs/97296c1a2aa5705046eb53ad742c9a9d598e9ce1/781d1414691d728d4ca09969924d5a5d8f0c34f8/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2022/11/14 16:55:18 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'da_sklearn_lr' already exists. Creating a new version of this model...
2022/11/14 16:55:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: da_sklearn_lr, version 22
Created version '22' of model 'da_sklearn_lr'.
/Users/da/.cache/pants/named_caches/pex_root/venvs/97296c1a2aa5705046eb53ad742c9a9d598e9ce1/781d1414691d728d4ca09969924d5a5d8f0c34f8/lib/python3.8/site-packages/rikai/spark/sql/codegen/mlflow_logger.py:183

## Step 4: Create the model using the registered MLflow uri

In [5]:
from rikai.spark.sql.codegen.mlflow_logger import CONF_MLFLOW_TRACKING_URI
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set(CONF_MLFLOW_TRACKING_URI, mlflow_tracking_uri)
spark.sql(f"""
CREATE OR REPLACE MODEL mlflow_sklearn_m USING 'mlflow:///{registered_model_name}';
"""
)

spark.sql("show models").show(1, vertical=False, truncate=False)


+----------------+------+-----------------------+-------+
|name            |flavor|uri                    |options|
+----------------+------+-----------------------+-------+
|mlflow_sklearn_m|      |mlflow:///da_sklearn_lr|       |
+----------------+------+-----------------------+-------+



## Step 5: predict using the registered Rikai model

In [6]:
result = spark.sql(f"""
select
  ML_PREDICT(mlflow_sklearn_m, array(0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
        -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613))
"""
)

result.printSchema()
result.toPandas()

root
 |-- mlflow_sklearn_m: float (nullable = true)



,mlflow_sklearn_m
0,206.116684


In [7]:
spark.sql("""
select  ML_PREDICT(mlflow_sklearn_m, array(-0.00188202, -0.04464164, -0.05147406, -0.02632753, -0.00844872,
        -0.01916334,  0.07441156, -0.03949338, -0.06833155, -0.09220405))
""").toPandas()

,mlflow_sklearn_m
0,68.07103
